In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

In [ ]:
!wget https://raw.githubusercontent.com/apache/spark/master/data/mllib/sample_libsvm_data.txt

In [ ]:
from pyspark.mllib.util import MLUtils

data = MLUtils.loadLibSVMFile(sc, 'sample_libsvm_data.txt')

In [ ]:
data.take(1)[0]

## Standard Scaler

In [ ]:
from pyspark.mllib.feature import StandardScaler, StandardScalerModel
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint

label = data.map(lambda x: x.label)
features = data.map(lambda x: x.features)

scaler1 = StandardScaler().fit(features)
scaler2 = StandardScaler(withMean=True, withStd=True).fit(features)

scaler_data1 = label.zip(scaler1.transform(features))

scaler_data2 = label.zip(scaler2.transform(features.map(lambda x: Vectors.dense(x.toArray()))))

In [ ]:
scaler_data1.take(1)[0]

In [ ]:
scaler_data2.take(1)[0]

In [ ]:
scaler_data1.map(lambda x: LabeledPoint(x[0], x[1])).take(1)

## Normalizer

In [ ]:
from pyspark.mllib.feature import Normalizer

labels = data.map(lambda x: x.label)
features = data.map(lambda x: x.features)

normalizer1 = Normalizer()
normalizer2 = Normalizer(p=float("inf"))

norm_data1 = labels.zip(normalizer1.transform(features))

norm_data2 = labels.zip(normalizer2.transform(features))

In [ ]:
norm_data1.take(1)[0]

In [ ]:
norm_data2.take(1)[0]

## PCA

In [ ]:
from pyspark.mllib.feature import PCA

model = PCA(10).fit(scaler_data1.map(lambda x: Vectors.dense(x[1].toArray())))
pc = model.transform(scaler_data1.map(lambda x: Vectors.dense(x[1].toArray())))
pc.take(5)

In [ ]:
sc.stop()